In [1]:
import io
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import os
import re
import tensorflow as tf
import time
import unicodedata

from sklearn.model_selection import train_test_split

##### Gpu acceleration and reduced memory usage

In [ ]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
tf.config.set_soft_device_placement(True)

#### Data load

We'll use a language dataset provided by http://www.manythings.org/anki/

In [2]:
# Загружаем файл
path_to_file = "rus.txt"

In [3]:
#просмотр файла
with open(path_to_file, encoding="utf-8") as f:
    read_data = f.readline()
read_data

'Go.\tМарш!\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1159202 (shanghainese)\n'

#### Data preprocessing

In [4]:
#функция препроцессинга
def preprocess_sentence(w):
  #переводим предложение к нижнему регистру и удалем начальные и конечные пробелы
    w = w.lower().strip()

  # отделяем пробелом слово и следующую за ним пунктуацию
  # пример: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

  # все, кроме букв и знаков пунктуации, заменяем пробелом
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)
  
  #удаляем лишние пробелы в начале и конце
    w = w.strip()

  # создаем начало и конец последовательности
  # теперь модель знает, где начинать и заканчивать предсказания
    w = '<start> ' + w + ' <end>'
    return w

In [5]:
#пример работы препроцессинга
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

#### Create dataset

In [6]:
# 1. Убираем акценты
# 2. Очищаем предложения
# 3. Возвращаем пары слов: [ENG, RUS]
def create_dataset(path, num_examples):
  #считываем строки файла
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  #каждую строку разделяем на пробелы, берем первые 2 слова, препроцессим их и возвращаем пару
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [7]:
#пример работы
en, ru = create_dataset(path_to_file, None)
print(en[0])
print(ru[0])

<start> go . <end>
<start> марш ! <end>


In [8]:
# количество данных в датасете
len(en), len(ru)

(479223, 479223)

#### Dataset loader

In [9]:
def tokenize(lang):
      #токенизируем текст, отфильтвовываем пробелы
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
      #обновляем внутренний словарь на основе lang
    lang_tokenizer.fit_on_texts(lang)
      #преобразуем каждый элемент из lang в последовательность чисел
    tensor = lang_tokenizer.texts_to_sequences(lang)
      #преобразуем тензор в матрицу (кол-во тензоров * max-длина), 
      #при этом короткие последовательности заполняем нулями сзади, а длинные -- обрезаем сзади
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

#### Dataset creation

In [10]:
def load_dataset(path, num_examples=None):
      # создаем очищенные анг (выходные), русские (входные) пары
    targ_lang, inp_lang = create_dataset(path, num_examples)
    #применяем токенизацию к каждому элементы из пары
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [11]:
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

input_tensor, target_tensor

(array([[   1, 5696,   22, ...,    0,    0,    0],
        [   1,  189,    3, ...,    0,    0,    0],
        [   1,  282,    3, ...,    0,    0,    0],
        ...,
        [   1,    6,   21, ...,    0,    0,    0],
        [   1,   18,    7, ...,    0,    0,    0],
        [   1,    6,   21, ...,    0,    0,    0]]),
 array([[ 1, 27,  3, ...,  0,  0,  0],
        [ 1, 27,  3, ...,  0,  0,  0],
        [ 1, 27,  3, ...,  0,  0,  0],
        ...,
        [ 1,  5, 16, ...,  0,  0,  0],
        [ 1,  5, 16, ...,  0,  0,  0],
        [ 1,  5, 16, ...,  0,  0,  0]]))

In [12]:
# Вычисляем максимальную длину тензоров
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
max_length_targ, max_length_inp

(11, 15)

In [13]:
# Создаем тренировочные и валидационные датасеты
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# размеры датасетов
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [14]:
input_tensor_train, target_tensor_train

(array([[   1,    6,  260, ...,    0,    0,    0],
        [   1,    4, 9671, ...,    0,    0,    0],
        [   1,  560,   12, ...,    0,    0,    0],
        ...,
        [   1,   13,   53, ...,    0,    0,    0],
        [   1,   12, 1657, ...,    0,    0,    0],
        [   1, 5072, 5912, ...,    0,    0,    0]]),
 array([[   1,    5,  524, ...,    0,    0,    0],
        [   1,    4, 4460, ...,    0,    0,    0],
        [   1,   47,   15, ...,    0,    0,    0],
        ...,
        [   1,   19,   65, ...,    0,    0,    0],
        [   1,   22,    7, ...,    0,    0,    0],
        [   1,   24, 2252, ...,    0,    0,    0]]))

In [15]:
#функция получения из токена текста (выводим токен и его индекс)
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [16]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
6 ----> том
260 ----> говорит
9 ----> ,
17 ----> что
19 ----> он
15 ----> в
183 ----> порядке
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
5 ----> tom
524 ----> says
124 ----> he's
439 ----> fine
3 ----> .
2 ----> <end>


In [17]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
#количество эпох
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
#из каждого элемента (input_tensor_train, target_tensor_train) создает тензор
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
#разбиваем датасет на батчи (списки по 64), удаляя последний неполный батч
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [18]:
example_input_batch, example_target_batch = next(iter(dataset))
print(example_input_batch.shape, example_target_batch.shape)
example_input_batch[0], example_target_batch[0]

(64, 15) (64, 11)


(<tf.Tensor: shape=(15,), dtype=int32, numpy=
 array([   1, 6387,  261, 1913,   49,    3,    2,    0,    0,    0,    0,
           0,    0,    0,    0])>,
 <tf.Tensor: shape=(11,), dtype=int32, numpy=array([  1, 406, 171,  61,  15,   3,   2,   0,   0,   0,   0])>)

#### Machine translation model building

**Encoder**

In [19]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=False,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return state

    def initialize_hidden_state(self):
    #создаем тензор из нулей размера (батч, кол-во ячеек)
        return tf.zeros((self.batch_sz, self.enc_units))

In [20]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# инициализитеруем начальное скрытое состояние (из нулей)
sample_hidden = encoder.initialize_hidden_state()
# применяем энкодер к входному батчу и скрытому состоянию
sample_hidden = encoder(example_input_batch, sample_hidden)
# print ('Форма выхода энкодера: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (64, 1024)


In [21]:
sample_hidden

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[-1.20029990e-02,  1.66133360e-03, -6.31120129e-05, ...,
        -1.43919522e-02,  1.44618331e-02, -8.83039646e-03],
       [-1.19978283e-02,  1.78099168e-03, -1.80959658e-04, ...,
        -1.45703126e-02,  1.48877455e-02, -8.86684842e-03],
       [-1.19848596e-02,  1.80697208e-03, -1.44916470e-04, ...,
        -1.47417625e-02,  1.48633337e-02, -8.91372934e-03],
       ...,
       [-1.19560324e-02,  1.79350632e-03,  2.82718029e-05, ...,
        -1.41730700e-02,  1.44670717e-02, -8.68918840e-03],
       [-1.21374270e-02,  1.78678287e-03, -8.99007937e-05, ...,
        -1.45864338e-02,  1.47088431e-02, -8.83869920e-03],
       [-1.19473143e-02,  1.84538972e-03, -8.22641668e-05, ...,
        -1.46181956e-02,  1.46132344e-02, -8.85110255e-03]], dtype=float32)>

**Decoder**

In [22]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden):
        # x shape после прохождения через эмбеддинг == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # отправляем в GRU входные данные и скрытое состояние (от энкодера)
        #выход GRU (batch_size, timesteps, units)
        #размер возвращаемого внутреннего состояния (batch_size, units)
        output, state = self.gru(x, initial_state=hidden)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # x shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state

In [23]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
#применяем декодер к случайному батчу из равномерного распределения (батч,1) и выходу энкодера
decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden)
decoder_sample_x.shape, decoder_sample_h.shape

(TensorShape([64, 7386]), TensorShape([64, 1024]))

In [24]:
decoder_sample_x

<tf.Tensor: shape=(64, 7386), dtype=float32, numpy=
array([[ 0.00777631,  0.0015943 , -0.00565019, ...,  0.00313168,
         0.00354554,  0.0038535 ],
       [ 0.00781316,  0.00156634, -0.00558414, ...,  0.00325799,
         0.00362245,  0.0038099 ],
       [ 0.00784024,  0.00148141, -0.00558086, ...,  0.00323906,
         0.003685  ,  0.00381215],
       ...,
       [ 0.00777819,  0.00156104, -0.00565952, ...,  0.00320803,
         0.00351055,  0.003876  ],
       [ 0.00780508,  0.00153168, -0.00561464, ...,  0.0031972 ,
         0.00362976,  0.00383433],
       [ 0.00781009,  0.00153064, -0.00559916, ...,  0.00319188,
         0.00363257,  0.00384646]], dtype=float32)>

#### Model compilling

In [25]:
#оптимизатор
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

#функция потерь
def loss_function(real, pred):
      #делаем инверсию значений сравнения каждого из real с нулем (возвращается true или false)
    mask = tf.math.logical_not(tf.math.equal(real, 0))
      #применяем функцию ошибок к реальным данным и предсказанным
    loss_ = loss_object(real, pred)
      #приводим тензор mask к новому типу loss_.dtype
    mask = tf.cast(mask, dtype=loss_.dtype)
      #умножаем loss_ на mask
    loss_ *= mask
      # возвращаем среднее значениe всех элементов
    return tf.reduce_mean(loss_)

**Сheckpoint**

In [26]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

#### Train model

In [27]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
  # перечисляем операции для автоматического дифференцирования
    with tf.GradientTape() as tape:
        #получаем выход encoder
        enc_hidden = encoder(inp, enc_hidden)
        #помещаем его в скрытое состояние decoder
        dec_hidden = enc_hidden
        #формируем вход декодера:
                 # берем список длины батч из индексов тега <start> (1)
                 # приписываем списку размерность 1 сзади (батч, 1)
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - выводим target в качестве следующего входа
        for t in range(1, targ.shape[1]):
          # помещаем enc_output в decoder
            predictions, dec_hidden = decoder(dec_input, dec_hidden)
          # считаем функцию потерь 
            loss += loss_function(targ[:, t], predictions)
          # используем teacher forcing (приписываем списку размерность 1 сзади)
          #посылаем dec_input на вход декордера 
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))
    #переменные
    variables = encoder.trainable_variables + decoder.trainable_variables
    #вычисляем градиенты loss по variables
    gradients = tape.gradient(loss, variables)
    #оптимизатор применяет подсчитанные градиенты
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [29]:
EPOCHS = 5

for epoch in range(EPOCHS):
    start = time.time()

  #инициализируем входное скрытое состояние (из нулей) размера (батч, кол-во рекуррентных ячеек)
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        #делаем шаг обучения. находим оштбку за этоху
        batch_loss = train_step(inp, targ, enc_hidden)
        #считаем ошибку
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                       batch,
                                                       batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.3908
Epoch 1 Batch 100 Loss 0.3011
Epoch 1 Batch 200 Loss 0.3685
Epoch 1 Batch 300 Loss 0.3820
Epoch 1 Batch 400 Loss 0.4277
Epoch 1 Batch 500 Loss 0.3743
Epoch 1 Batch 600 Loss 0.4398
Epoch 1 Batch 700 Loss 0.3606
Epoch 1 Batch 800 Loss 0.4213
Epoch 1 Batch 900 Loss 0.3317
Epoch 1 Batch 1000 Loss 0.3709
Epoch 1 Batch 1100 Loss 0.3263
Epoch 1 Batch 1200 Loss 0.4087
Epoch 1 Loss 0.3919
Time taken for 1 epoch 327.7747747898102 sec

Epoch 2 Batch 0 Loss 0.2616
Epoch 2 Batch 100 Loss 0.2309
Epoch 2 Batch 200 Loss 0.2099
Epoch 2 Batch 300 Loss 0.2204
Epoch 2 Batch 400 Loss 0.2017
Epoch 2 Batch 500 Loss 0.2447
Epoch 2 Batch 600 Loss 0.2209
Epoch 2 Batch 700 Loss 0.2598
Epoch 2 Batch 800 Loss 0.2371
Epoch 2 Batch 900 Loss 0.2182
Epoch 2 Batch 1000 Loss 0.3098
Epoch 2 Batch 1100 Loss 0.2084
Epoch 2 Batch 1200 Loss 0.2756
Epoch 2 Loss 0.2294
Time taken for 1 epoch 344.6755602359772 sec

Epoch 3 Batch 0 Loss 0.1246
Epoch 3 Batch 100 Loss 0.1633
Epoch 3 Batch 200 Loss 0.125

#### Translate
* Функция оценки аналогична циклу обучения, за исключением того, что здесь мы не используем *teacher forcing*. Входным сигналом для декодера на каждом временном шаге являются его предыдущие предсказания вместе со скрытым состоянием и выходным сигналом энкодера.
* Предсказания модели прекращаются, когда модель предскажет *end token*.
* Сохраняем *веса внимания для каждого временного шага*.

Примечание: Выходной сигнал энкодера вычисляется только один раз для одного входа.

In [30]:
def evaluate(sentence):
  #препоцессим предложение
    sentence = preprocess_sentence(sentence)
      #разбиваем предложение по пробелам и составляем список индексов каждого слова
    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
      #добиваем inputs нулями справа до максимальной длины входного текста
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                             maxlen=max_length_inp,
                                                             padding='post')
      #преобразуем inputs в тензор
    inputs = tf.convert_to_tensor(inputs)

    result = ''
      # инициализируем входной хидден из нулей размера (1, units)
    hidden = [tf.zeros((1, units))]
      #подаем inputs и hidden в encoder
    enc_hidden = encoder(inputs, hidden)

      #инициализируем входной хидден декодера -- выходной хидден энкодера
    dec_hidden = enc_hidden
      #вход декодера -- список [индекс start] размера(1,1)
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
            #получаем выход декодера
        predictions, dec_hidden = decoder(dec_input, dec_hidden)

        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '

   #заканчиваем на токене end
        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence

    # предсказанный predicted ID подаем обратно в декодер (размер (1,1))
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [31]:
#функция перевода
def translate(sentence):
    result, sentence = evaluate(sentence)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [33]:
# загружаем последний checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [34]:
translate('Моросит')

Input: <start> моросит <end>
Predicted translation: it's completely dark . <end> 


In [36]:
translate('Идет дождь')

Input: <start> идет дождь <end>
Predicted translation: it's raining . <end> 


In [37]:
translate('Завтра мне идти на работу')

Input: <start> завтра мне идти на работу <end>
Predicted translation: i'll go back to my job . <end> 


In [38]:
translate('Завтра не наступит никогда')

Input: <start> завтра не наступит никогда <end>
Predicted translation: tomorrow won't be fixed . <end> 


In [39]:
translate('Во дворе трава, на траве дрова')

Input: <start> во дворе трава , на траве дрова <end>
Predicted translation: all of us is in french . <end> 


In [40]:
translate('Что делать если ничего не хочется?')

Input: <start> что делать если ничего не хочется ? <end>
Predicted translation: what do i say no ? <end> 


In [57]:
translate('Перевод предложения зависит от словарь на котором построена модель')

Input: <start> перевод предложения зависит от словарь на котором построена модель <end>
Predicted translation: the feud is over . <end> 


#### Вывод: 
Из-за малого словаря имеется ряд неточностей в переводе, также наблюдается ряд неточностей при переводе длинных предложений. Это обусловлено отсутствием "внимания".